# Machine Learning parte 2: Otimização com exploração aleatória

* Este estudo é uma continuação do estudo de [ML parte 1: Otimização de modelos através de hiperparâmetros](https://colab.research.google.com/drive/1zkmLyzgb1jqOybg_ZRvQR9_uk0_AIYHr?usp=sharing).

## Abertura do Dataset

In [1]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [2]:
# simulação de uma péssima organização dos dados
dados_azar = dados.sort_values('vendido', ascending = True)
x_azar = dados_azar[['preco', 'idade_do_modelo', 'km_por_ano']]
y_azar = dados_azar['vendido']

# Busca aleatória (Randomized Search CV)

* Define-se um espaço de parâmetros e uma quantidade de iterações que o algoritmo deve executar, menor do que o espaço completo.

* A exploração é feita a quantidade de vezes definida, portanto nem todo o espaço será explorado, reduzindo o custo do processo.

* Como definido no código abaixo, o espaço de busca resulta em, no máximo, 32 variações de modelo. Como a escolha será aleatória, nem todos serão explorados.

In [6]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold

SEED = 301
np.random.seed(SEED)

espaco_de_parametros = { 
    "max_depth" : [3, 5],
    "min_samples_split": [32, 64, 128],
    "min_samples_leaf": [32, 64, 128],
    "criterion": ["gini", "entropy"]
}

busca = RandomizedSearchCV(DecisionTreeClassifier(),
                    espaco_de_parametros,
                    cv = KFold(n_splits = 5, shuffle=True),
                    random_state = SEED,
                    n_iter = 16) # apenas 16 variações serão exploradas, ao invés de 36

busca.fit(x_azar, y_azar)
resultados = pd.DataFrame(busca.cv_results_)

* O treino foi feito com cross-validation, então a validação será da mesma forma.

In [13]:
def imprime_score(scores):
  media = scores.mean()
  desvio_padrao = scores.std()

  print(f'Accuracy mean = {media*100:.2f}%')
  print(f'Accuracy interval = [{(media - 2*desvio_padrao)*100 :.2f}%, {(media + 2*desvio_padrao)*100 :.2f}%]')

In [14]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(busca, x_azar, y_azar, cv = KFold(n_splits=5, shuffle=True))

In [15]:
imprime_score(scores)

Accuracy mean = 78.69%
Accuracy interval = [77.43%, 79.95%]


In [16]:
melhor = busca.best_estimator_
print(melhor)

DecisionTreeClassifier(max_depth=3, min_samples_leaf=32, min_samples_split=64)


* O estimador encontrado pela busca aleatória foi semelhante ao encontrado no estudo anterior, com uma diferença no min_samples_split.

* O tempo gasto para encontrar esta solução foi muito menor, apesar a diferença pequena.

## Espaço de parâmetros mais abrangente

In [17]:
from scipy.stats import randint

SEED = 301
np.random.seed(SEED)

espaco_de_parametros = { 
    "max_depth" : [3, 5, 10, 15, 20, 30, None],
    "min_samples_split": randint(32, 129), # vai gerando números aleatórios à medida que são requisitados
    "min_samples_leaf": randint(32, 129),  # não apenas uma única vez
    "criterion": ["gini", "entropy"]
}

busca = RandomizedSearchCV(DecisionTreeClassifier(),
                    espaco_de_parametros,
                    cv = KFold(n_splits = 5, shuffle=True),
                    random_state = SEED,
                    n_iter = 16) # apenas 16 variações serão exploradas, ao invés de 36

busca.fit(x_azar, y_azar)
resultados = pd.DataFrame(busca.cv_results_)

In [18]:
scores = cross_val_score(busca, x_azar, y_azar, cv = KFold(n_splits=5, shuffle=True))
imprime_score(scores)

Accuracy mean = 78.71%
Accuracy interval = [77.49%, 79.93%]


In [19]:
melhor = busca.best_estimator_
print(melhor)

DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_leaf=71,
                       min_samples_split=100)


* A acurácia média foi um pouco maior e o intervalo mais afunilado, em comparação com o estimador anterior.

* Somente um hiperparâmetro permaneceu, a profundidade máxima 3.